# Data

> Datasets and collators for Extreme Classification

In [ ]:
#| default_exp data

In [ ]:
#| hide
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from scipy import sparse
import torch, inspect, numpy as np, pandas as pd
from IPython.display import display
from typing import Dict, Optional, Callable
from torch.utils.data import Dataset,DataLoader
from xclib.data import data_utils as du
from transformers import PreTrainedTokenizerBase, AutoTokenizer

from fastcore.utils import *
from fastcore.meta import *
from fastcore.dispatch import *

from xcai.core import *

In [ ]:
#| hide
from xcai.transform import *

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Data

In [ ]:
dset_dir = f'/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K'
data_cfg = {
    'cols': ['identifier', 'input_text'],
    'use_tokz': True,
    'tokz': 'bert-base-uncased',
    'fld': 'input_text',
    'max_len': 32,
}

In [ ]:
#| export
class MainXCData:
    
    @classmethod
    @delegates(Info.from_txt)
    def from_file(cls, data_lbl:str, data_info:str, lbl_info:str, data_lbl_filterer:Optional[str]=None, **kwargs):
        return {
            'data_lbl': du.read_sparse_file(data_lbl),
            'data_info': Info.from_txt(data_info, **kwargs),
            'lbl_info': Info.from_txt(lbl_info, **kwargs),
            'data_lbl_filterer': Filterer.load_filter(data_lbl_filterer),
        }
    

In [ ]:
train_cfg = {
    'data_lbl': f'{dset_dir}/trn_X_Y.txt',
    'data_info': f'{dset_dir}/raw_data/train.raw.txt',
    'lbl_info': f'{dset_dir}/raw_data/label.raw.txt',
    'data_lbl_filterer': f'{dset_dir}/filter_labels_train.txt'
}
train_data = MainXCData.from_file(**train_cfg, **data_cfg)

In [ ]:
#| export
class MetaXCData:
    
    @classmethod
    @delegates(Info.from_txt)
    def from_file(cls, data_meta:str, lbl_meta:str, meta_info:str, prefix:str, **kwargs):
        return {
            'prefix': prefix,
            'data_meta': du.read_sparse_file(data_meta),
            'lbl_meta': du.read_sparse_file(lbl_meta),
            'meta_info': Info.from_txt(meta_info, **kwargs),
        }
    

In [ ]:
meta_cfg = {
    'prefix': 'hlk',
    'data_meta': f'{dset_dir}/hyper_link_trn_X_Y.txt',
    'lbl_meta': f'{dset_dir}/hyper_link_lbl_X_Y.txt',
    'meta_info': f'{dset_dir}/raw_data/hyper_link.raw.txt',
}
meta_data = MetaXCData.from_file(**meta_cfg, **data_cfg)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


## Dataset

### `BaseXCDataset`

In [ ]:
#| export
class BaseXCDataset(Dataset):
    def __init__(self):
        self.n_data, self.n_lbl, self.n_meta, self.n_samples = None, None, None, None

    def __len__(self):
        return self.n_data if self.n_data is not None else 0

    def splitter(self, valid_pct:Optional[float]=0.2, seed=None):
        if seed is not None: torch.manual_seed(seed)
        rnd_idx = list(torch.randperm(self.n_data).numpy())
        cut = int(valid_pct * self.n_data)
        train, valid = self._getitems(rnd_idx[cut:]), self._getitems(rnd_idx[:cut])
        return train, valid
    
    def _verify_info(self, info:Dict):
        if info is None: raise ValueError('`info` cannot be empty.')
        n_info = [len(v) for k,v in info.items()]
        if len(n_info) == 0 or n_info[0] == 0: raise ValueError('`info` cannot be empty.')
        if np.all([n_info[0] == o for o in n_info]) == False: 
            raise ValueError('All `data_info` fields should have equal number of elements.')
        return n_info[0]
        
    def show_data(self, n:Optional[int]=10, seed:Optional[int]=None):
        if n < 1: return
        if seed: np.random.seed(seed)
        idx = np.random.permutation(self.n_data)[:n]
        d = [self[i] for i in idx]
        df = pd.DataFrame({k:[o[k] for o in d] for k in d[0]})
        with pd.option_context('display.max_colwidth', None, 'display.max_columns', None):
            display(df)
            

### `MainXCDataset`

In [ ]:
#| export
class MainXCDataset(BaseXCDataset):
    def __init__(self,
                 data_info:Dict,
                 data_lbl:Optional[sparse.csr_matrix]=None,
                 lbl_info:Optional[Dict]=None,
                 data_lbl_filterer:Optional[Union[sparse.csr_matrix,np.array]]=None,
                 n_samples:Optional[int]=None,
                 **kwargs):
        super().__init__()
        store_attr('data_info,data_lbl,lbl_info,data_lbl_filterer,n_samples')
        self._verify_inputs()
        
    @classmethod
    @delegates(MainXCData.from_file)
    def from_file(cls, n_samples:Optional[int]=None, **kwargs):
        return cls(**MainXCData.from_file(**kwargs), n_samples=n_samples)
        

In [ ]:
#| export
@patch
def _verify_inputs(cls:MainXCDataset):
    cls.n_data = cls._verify_info(cls.data_info)
    if cls.data_lbl is not None:
        if cls.n_data != cls.data_lbl.shape[0]:
            raise ValueError(f'`data_info`({cls.n_data}) should have same number of datapoints as `data_lbl`({cls.data_lbl.shape[0]})')
        cls.n_lbl = cls.data_lbl.shape[1]
        if cls.lbl_info is not None:
            n_lbl = cls._verify_info(cls.lbl_info)
            if n_lbl != cls.data_lbl.shape[1]:
                raise ValueError(f'`lbl_info`({n_lbl}) should have same number of labels as `data_lbl`({cls.data_lbl.shape[1]})')

In [ ]:
#| export
@patch
def __getitem__(cls:MainXCDataset, idx:int):
    x = {f'data_{k}': v[idx] for k,v in cls.data_info.items()}
    if cls.n_lbl is not None:
        prefix = 'lbl2data'
        x[f'{prefix}_idx'] = cls.data_lbl[idx].indices.tolist()
        if cls.n_samples: x[f'{prefix}_idx'] = [x[f'{prefix}_idx'][i] for i in np.random.permutation(len(x[f'{prefix}_idx']))[:cls.n_samples]]
        if cls.lbl_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in cls.lbl_info.items()})
    return x
    

In [ ]:
#| export
@patch
def __getitems__(cls:MainXCDataset, idxs:List):
    x = {f'data_{k}':[v[idx] for idx in idxs] for k,v in cls.data_info.items()}
    if cls.n_lbl is not None:
        prefix = 'lbl2data'
        x[f'{prefix}_idx'] = [cls.data_lbl[idx].indices.tolist() for idx in idxs]
        if cls.n_samples: x[f'{prefix}_idx'] = [[o[i] for i in np.random.permutation(len(o))[:cls.n_samples]] for o in x[f'{prefix}_idx']]
        x[f'{prefix}_data2ptr'] = [len(o) for o in x[f'{prefix}_idx']]
        x[f'{prefix}_idx'] = list(chain(*x[f'{prefix}_idx']))
        if cls.lbl_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in cls.lbl_info.items()})
        return x
        

In [ ]:
#| export
@patch
def _getitems(cls:MainXCDataset, idxs:List):
    return MainXCDataset(
        {k:[v[idx] for idx in idxs] for k,v in cls.data_info.items()}, 
        cls.data_lbl[idxs] if cls.data_lbl is not None else None, 
        cls.lbl_info, n_samples=cls.n_samples
    )

#### Example

In [ ]:
train_main = MainXCDataset(**train_data, n_samples=2)

In [ ]:
train_main.show_data(n=3, seed=10)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask
0,Certificate_of_Formula_Compliance,Certificate of Formula Compliance,"[101, 8196, 1997, 5675, 12646, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",[244514],[Certificate_of_Origin],[Certificate of Origin],"[[101, 8196, 1997, 4761, 102]]","[[0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1]]"
1,2017_Sunwolves_season,2017 Sunwolves season,"[101, 2418, 3103, 12155, 20899, 2161, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[163257, 308041]","[Sunwolves, 2017_Super_Rugby_season]","[Sunwolves, 2017 Super Rugby season]","[[101, 3103, 12155, 20899, 102], [101, 2418, 3565, 4043, 2161, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]"
2,Hadza_people,Hadza people,"[101, 2018, 4143, 2111, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]","[8411, 164636]","[Twa_peoples, Aka_people]","[Twa peoples, Aka people]","[[101, 1056, 4213, 7243, 102], [101, 9875, 2111, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1]]"


In [ ]:
split1, split2 = train_main.splitter()
split1.n_data, split2.n_data, split1.n_data+split2.n_data, train_main.n_data

(554466, 138616, 693082, 693082)

In [ ]:
train_main = MainXCDataset.from_file(**train_cfg, **data_cfg)

In [ ]:
train_main.show_data(n=3, seed=10)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask
0,Certificate_of_Formula_Compliance,Certificate of Formula Compliance,"[101, 8196, 1997, 5675, 12646, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",[244514],[Certificate_of_Origin],[Certificate of Origin],"[[101, 8196, 1997, 4761, 102]]","[[0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1]]"
1,2017_Sunwolves_season,2017 Sunwolves season,"[101, 2418, 3103, 12155, 20899, 2161, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[163257, 308041]","[Sunwolves, 2017_Super_Rugby_season]","[Sunwolves, 2017 Super Rugby season]","[[101, 3103, 12155, 20899, 102], [101, 2418, 3565, 4043, 2161, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]"
2,Hadza_people,Hadza people,"[101, 2018, 4143, 2111, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]","[8411, 164636, 230647]","[Twa_peoples, Aka_people, Bushmen]","[Twa peoples, Aka people, Bushmen]","[[101, 1056, 4213, 7243, 102], [101, 9875, 2111, 102], [101, 5747, 3549, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]"


### `MetaXCDataset`

In [ ]:
#| export
class MetaXCDataset(BaseXCDataset):

    def __init__(self,
                 prefix:str,
                 data_meta:sparse.csr_matrix, 
                 lbl_meta:sparse.csr_matrix, 
                 meta_info:Optional[Dict]=None, 
                 n_samples:Optional[int]=None, 
                 **kwargs):
        store_attr('prefix,data_meta,lbl_meta,meta_info,n_samples')
        self._verify_inputs()

    def _getitems(self, idxs:List):
        return MetaXCDataset(self.prefix, self.data_meta[idxs], self.lbl_meta, self.meta_info, self.n_samples)
        
    @classmethod
    @delegates(MetaXCData.from_file)
    def from_file(cls, n_samples:Optional[int]=None, **kwargs):
        return cls(**MetaXCData.from_file(**kwargs), n_samples=n_samples)

    @typedispatch
    def get_lbl_meta(self, idx:int):
        prefix = f'{self.prefix}2lbl2data'
        x = {f'{prefix}_idx': self.lbl_meta[idx].indices.tolist()}
        if self.n_samples: x[f'{prefix}_idx'] = [x[f'{prefix}_idx'][i] for i in np.random.permutation(len(x[f'{prefix}_idx']))[:self.n_samples]]
        if self.meta_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in self.meta_info.items()})
        return x
    
    @typedispatch
    def get_lbl_meta(self, idxs:List):
        prefix = f'{self.prefix}2lbl2data'
        x = {f'{prefix}_idx': [self.lbl_meta[idx].indices.tolist() for idx in idxs]}
        if self.n_samples: x[f'{prefix}_idx'] = [[o[i] for i in np.random.permutation(len(o))[:self.n_samples]] for o in x[f'{prefix}_idx']]
        if self.meta_info is not None:
            x.update({f'{prefix}_{k}':[[v[i] for i in o] for o in x[f'{prefix}_idx']] for k,v in self.meta_info.items()})
        return x
        
    def get_data_meta(self, idx:int):
        prefix = f'{self.prefix}2data'
        x = {f'{prefix}_idx': self.data_meta[idx].indices.tolist()}
        if self.n_samples: x[f'{prefix}_idx'] = [x[f'{prefix}_idx'][i] for i in np.random.permutation(len(x[f'{prefix}_idx']))[:self.n_samples]]
        if self.meta_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in self.meta_info.items()})
        return x

    def shape(self):
        return (self.n_data, self.n_lbl, self.n_meta)
        
    def show_data(self, is_lbl:Optional[bool]=False, n:Optional[int]=10, seed:Optional[int]=None):
        if n < 1: return
        if seed: np.random.seed(seed)
        idx = np.random.permutation(self.n_lbl if is_lbl else self.n_data)[:n]
        d = [self.get_lbl_meta(int(i)) for i in idx] if is_lbl else [self.get_data_meta(i) for i in idx]
        df = pd.DataFrame({k:[o[k] for o in d] for k in d[0]})
        with pd.option_context('display.max_colwidth', None):
            display(df)
    

If metadata is available for the main task, then there will be `data_meta` and `lbl_meta`.

In [ ]:
#| export
@patch
def _verify_inputs(cls:MetaXCDataset):
    cls.n_data, cls.n_lbl, cls.n_meta = cls.data_meta.shape[0], cls.lbl_meta.shape[0], cls.data_meta.shape[1]
    if cls.lbl_meta.shape[1] != cls.n_meta:
        raise ValueError(f'`lbl_meta`({cls.lbl_meta.shape[1]}) should have same number of columns as `data_meta`({cls.n_meta}).')
    if cls.meta_info is not None:
        n_meta = cls._verify_info(cls.meta_info)
        if n_meta != cls.n_meta:
            raise ValueError(f'`meta_info`({n_meta}) should have same number of entries as number of columns of `data_meta`({cls.n_meta})')
            

#### Example

In [ ]:
train_meta = MetaXCDataset(**meta_data, n_samples=3)

In [ ]:
train_meta.show_data(n=3)

,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask
0,"[1188675, 186761, 1188713]","[Thomas_Howard_Fellows, Eric_Verdonk, Martin_Studach]","[Thomas Howard Fellows, Eric Verdonk, Martin Studach]","[[101, 2726, 4922, 13572, 102], [101, 4388, 2310, 28176, 2243, 102], [101, 3235, 16054, 6776, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]"
1,"[484231, 39074, 1362879]","[Geographic_Names_Information_System, United_States_Census_Bureau, List_of_cities_and_towns_in_Arizona]","[Geographic Names Information System, United States Census Bureau, List of cities and towns in Arizona]","[[101, 9183, 3415, 2592, 2291, 102], [101, 2142, 2163, 2883, 4879, 102], [101, 2862, 1997, 3655, 1998, 4865, 1999, 5334, 102]]","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]"
2,"[733563, 145339, 1442571]","[Gene_mapping, Proceedings_of_the_National_Academy_of_Sciences_of_the_United_States_of_America, Marc_Rieffel]","[Gene mapping, Proceedings of the National Academy of Sciences of the United States of America, Marc Rieffel]","[[101, 4962, 12375, 102], [101, 8931, 1997, 1996, 2120, 2914, 1997, 4163, 1997, 1996, 2142, 2163, 1997, 2637, 102], [101, 7871, 15544, 12879, 7959, 2140, 102]]","[[0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]"


In [ ]:
split1, split2 = train_meta.splitter()
split1.shape(), split2.shape()

((554466, 312330, 2458399), (138616, 312330, 2458399))

In [ ]:
train_meta.show_data(n=3, is_lbl=True)

,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,"[25125, 11543, 49413]","[Newspaper, Catholic_Church, Romandy]","[Newspaper, Catholic Church, Romandy]","[[101, 3780, 102], [101, 3234, 2277, 102], [101, 3142, 5149, 102]]","[[0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]"
1,"[657604, 507661, 163502]","[Niklas_Luhmann, JÃ¼rgen_Habermas, Talcott_Parsons]","[Niklas Luhmann, JÃ¼rgen Habermas, Talcott Parsons]","[[101, 23205, 8523, 11320, 13890, 2078, 102], [101, 14855, 29664, 25892, 5292, 5677, 9335, 102], [101, 21368, 13124, 13505, 102]]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]"
2,"[112522, 592260, 22599]","[1960_Winter_Olympics, Frensham_School, Order_of_the_British_Empire]","[1960 Winter Olympics, Frensham School, Order of the British Empire]","[[101, 3624, 3467, 3783, 102], [101, 10424, 6132, 3511, 2082, 102], [101, 2344, 1997, 1996, 2329, 3400, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]"


In [ ]:
train_meta.shape()

(693082, 312330, 2458399)

In [ ]:
train_meta = MetaXCDataset.from_file(**meta_cfg, **data_cfg)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
train_meta.show_data(n=3)

,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask
0,"[38, 122, 254, 281, 292, 697, 698, 701, 767, 2504, 2754, 2764, 2808, 3223, 3600, 4083, 4322, 4502, 4503, 5203, 19555, 26950, 35767, 45880, 57613, 189987, 247697, 260730, 262600, 280189, 297163, 297313, 361107, 416836, 471498, 619765, 666342, 719698, 818704, 1677742, 2037687, 2414194, 2414195, 2414196]","[Philippines, The_New_York_Times, United_States, France, Forbes, Netherlands, Colombia, Australia, Canada, South_Korea, Japan, Taiwan, New_York_Post, Thailand, Vice_president, Singapore, Food_and_Drug_Administration, Russell_2000_Index, Public_company, Los_Angeles_Times, Internal_Revenue_Service, Certified_Public_Accountant, Time_(magazine), Deseret_News, The_Salt_Lake_Tribune, Barry_Minkow, Chief_financial_officer, UniversitÃ©_de_MontrÃ©al, Therapeutic_Goods_Administration, Chief_operating_officer, U.S._Securities_and_Exchange_Commission, National_Business_Review, Naval_Postgraduate_School, San_Diego_Reader, Commerce_Commission, NSF_International, Public_relations_officer, United_States_Pharmacopeia, USANA_Amphitheatre, Herb_Greenberg, MonaVie, Fraud_Discovery_Institute, Denis_Waitley, Myron_W._Wentz]","[Philippines, The New York Times, United States, France, Forbes, Netherlands, Colombia, Australia, Canada, South Korea, Japan, Taiwan, New York Post, Thailand, Vice president, Singapore, Food and Drug Administration, Russell 2000 Index, Public company, Los Angeles Times, Internal Revenue Service, Certified Public Accountant, Time (magazine), Deseret News, The Salt Lake Tribune, Barry Minkow, Chief financial officer, UniversitÃ© de MontrÃ©al, Therapeutic Goods Administration, Chief operating officer, U.S. Securities and Exchange Commission, National Business Review, Naval Postgraduate School, San Diego Reader, Commerce Commission, NSF International, Public relations officer, United States Pharmacopeia, USANA Amphitheatre, Herb Greenberg, MonaVie, Fraud Discovery Institute, Denis Waitley, Myron W. Wentz]","[[101, 5137, 102], [101, 1996, 2047, 2259, 2335, 102], [101, 2142, 2163, 102], [101, 2605, 102], [101, 10822, 102], [101, 4549, 102], [101, 7379, 102], [101, 2660, 102], [101, 2710, 102], [101, 2148, 4420, 102], [101, 2900, 102], [101, 6629, 102], [101, 2047, 2259, 2695, 102], [101, 6504, 102], [101, 3580, 2343, 102], [101, 5264, 102], [101, 2833, 1998, 4319, 3447, 102], [101, 5735, 2456, 5950, 102], [101, 2270, 2194, 102], [101, 3050, 3349, 2335, 102], [101, 4722, 6599, 2326, 102], [101, 7378, 2270, 17907, 102], [101, 2051, 1006, 2932, 1007, 102], [101, 4078, 7869, 2102, 2739, 102], [101, 1996, 5474, 2697, 10969, 102], [101, 6287, 8117, 24144, 102], [101, 2708, 3361, 2961, 102], [101, 4895, 16402, 28032, 2050, 29652, 2139, 18318, 2527, 29652, 2389, 102], [101, 17261, 5350, 3447, 102], [101, 2708, 4082, 2961, 102], [101, 1057, 1012, 1055, 1012, 12012, 1998, 3863, 3222, 102], [101, 2120, 2449, 3319, 102], [101, 3987, 15438, 2082, 102], [101, 2624, 5277, 8068, 102], [101, 6236, 3222, 102], [101, 24978, 2546, 2248, 102], [101, 2270, 4262, 2961, 102], [101, 2142, 2163, 6887, 27292, 22684, 5051, 2401, 102], [101, 3915, 2532, 23713, 16584, 20192, 7913, 102], [101, 12810, 24190, 102], [101, 13813, 13469, 102], [101, 9861, 5456, 2820, 102], [101, 11064, 3524, 3051, 102], [101, 2026, 4948, 1059, 1012, 2253, 2480, 102]]","[[0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0

### `XCDataset`

In [ ]:
#| export
class MetaXCDatasets(dict):

    def __init__(self, meta:Dict):
        super().__init__(meta)
        for o in meta: setattr(self, o, meta[o])
        

In [ ]:
#| export
class XCDataset(BaseXCDataset):

    def __init__(self, data:MainXCDataset, **kwargs):
        super().__init__()
        self.data, self.meta = data, MetaXCDatasets({k:kwargs[k] for k in self.get_meta_args(**kwargs) if isinstance(kwargs[k], MetaXCDataset)})
        self._verify_inputs()

    def _getitems(self, idxs:List):
        return XCDataset(self.data._getitems(idxs), **{k:meta._getitems(idxs) for k,meta in self.meta.items()})

    @staticmethod
    def get_meta_args(**kwargs):
        return [k for k in kwargs if re.match(r'.*_meta$', k)]
        
    @classmethod
    @delegates(MainXCDataset.from_file)
    def from_file(cls, **kwargs):
        data = MainXCDataset.from_file(**kwargs)
        meta_kwargs = {o:kwargs.pop(o) for o in cls.get_meta_args(**kwargs)}
        meta = {k:MetaXCDataset.from_file(**v, **kwargs) for k,v in meta_kwargs.items()}
        return cls(data, **meta)

    def _verify_inputs(self):
        self.n_data, self.n_lbl = self.data.n_data, self.data.n_lbl
        if len(self.meta):
            self.n_meta = self.meta[list(self.meta.keys())[0]].n_meta
            for meta in self.meta.values():
                if meta.n_data != self.n_data: raise ValueError(f'`meta`({meta.n_data}) and `data`({self.n_data}) should have the same number of datapoints.')
                if self.n_lbl is not None and meta.n_lbl != self.n_lbl: 
                    raise ValueError(f'`meta`({meta.n_lbl}) and `data`({self.n_lbl}) should have the same number of labels.')
                if meta.n_meta != self.n_meta: raise ValueError(f'Every `meta`({meta.n_meta},{self.n_meta}) should have the same number of entries.')

    def __getitem__(self, idx:int):
        x = self.data[idx]
        if self.n_meta:
            for m in self.meta.values():
                x.update(m.get_data_meta(idx))
                if self.n_lbl: x.update(m.get_lbl_meta(x['lbl2data_idx']))
        return x

    @property
    def lbl_info(self): return self.data.lbl_info

    def one_batch(self, bsz:Optional[int]=10, seed:Optional[int]=None):
        if seed is not None: torch.manual_seed(seed)
        idxs = list(torch.randperm(len(self)).numpy())[:bsz]
        return [self[idx] for idx in idxs]
       

#### Example

In [ ]:
train_dset = XCDataset(train_main, hlk_meta=train_meta)

In [ ]:
b = train_dset.one_batch(10)

In [ ]:
b

[{'data_identifier': 'Orikabe_Station',
  'data_input_text': 'Orikabe Station',
  'data_input_ids': [101, 2030, 7556, 4783, 2276, 102],
  'data_token_type_ids': [0, 0, 0, 0, 0, 0],
  'data_attention_mask': [1, 1, 1, 1, 1, 1],
  'lbl2data_idx': [113248],
  'lbl2data_identifier': ['List_of_Railway_Stations_in_Japan'],
  'lbl2data_input_text': ['List of Railway Stations in Japan'],
  'lbl2data_input_ids': [[101, 2862, 1997, 2737, 3703, 1999, 2900, 102]],
  'lbl2data_token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0]],
  'lbl2data_attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]],
  'hlk2data_idx': [4170, 1596964, 1359126],
  'hlk2data_identifier': ['Japanese_National_Railways',
   'List_of_highways_numbered_284',
   'Ichinoseki_Station'],
  'hlk2data_input_text': ['Japanese National Railways',
   'List of highways numbered 284',
   'Ichinoseki Station'],
  'hlk2data_input_ids': [[101, 2887, 2120, 7111, 102],
   [101, 2862, 1997, 10292, 8597, 26871, 102],
   [101, 22564, 5740, 3366, 3211, 2276, 102]],
 

In [ ]:
train_dset.show_data(n=2)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,Portuguese_Paratroopers,Portuguese Paratroopers,"[101, 5077, 11498, 13181, 27342, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]","[151631, 183910]","[Airborne_forces, Portuguese_Air_Force]","[Airborne forces, Portuguese Air Force]","[[101, 10519, 2749, 102], [101, 5077, 2250, 2486, 102]]","[[0, 0, 0, 0], [0, 0, 0, 0, 0]]","[[1, 1, 1, 1], [1, 1, 1, 1, 1]]","[1082329, 701264, 281]","[Special_Operations_Troops_Centre, Tancos, France]","[Special Operations Troops Centre, Tancos, France]","[[101, 2569, 3136, 3629, 2803, 102], [101, 9092, 13186, 102], [101, 2605, 102]]","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]","[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1]]","[[321269, 152744, 7156], [205409, 111094, 1980540]]","[[Le_Muy, Indonesian_invasion_of_East_Timor, Benjamin_Franklin], [Airstrike, Dornier_Do_27, Culatra_Island]]","[[Le Muy, Indonesian invasion of East Timor, Benjamin Franklin], [Airstrike, Dornier Do 27, Culatra Island]]","[[[101, 3393, 14163, 2100, 102], [101, 9003, 5274, 1997, 2264, 19746, 102], [101, 6425, 5951, 102]], [[101, 14369, 18886, 3489, 102], [101, 2079, 28484, 2079, 2676, 102], [101, 12731, 20051, 2527, 2479, 102]]]","[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]]","[[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]]"
1,African_theology,African theology,"[101, 3060, 8006, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]","[228980, 7989]","[African_theology, Christianity_in_Africa]","[African theology, Christianity in Africa]","[[101, 3060, 8006, 102], [101, 7988, 1999, 3088, 102]]","[[0, 0, 0, 0], [0, 0, 0, 0, 0]]","[[1, 1, 1, 1], [1, 1, 1, 1, 1]]","[534154, 534150, 91924]","[John_S._Pobee, Musimbi_Kanyoro, Western_Christianity]","[John S. Pobee, Musimbi Kanyoro, Western Christianity]","[[101, 2198, 1055, 1012, 13433, 11306, 102], [101, 14163, 5332, 14905, 2072, 22827, 7677, 3217, 102], [101, 2530, 7988, 102]]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]","[[534152, 534154, 531359], [442884, 531315, 805]]","[[Kwesi_Dickson, John_S._Pobee, Christianity_in_Africa], [Athanasius_of_Alexandria, Southeast_Africa, Bible]]","[[Kwesi Dickson, John S. Pobee, Christianity in Africa], [Athanasius of Alexandria, Southeast Africa, Bible]]","[[[101, 6448, 2229, 2072, 22076, 102], [101, 2198, 1055, 1012, 13433, 11306, 102], [101, 7988, 1999, 3088, 102]], [[101, 2012, 15788, 24721, 1997, 10297, 102], [101, 4643, 3088, 102], [101, 6331, 102]]]","[[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]]","[[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1]]]"


In [ ]:
split1, split2 = train_dset.splitter()
split1.data.n_data, {k:m.shape() for k,m in split1.meta.items()}, split2.data.n_data, {k:m.shape() for k,m in split2.meta.items()}

(554466,
 {'hlk_meta': (554466, 312330, 2458399)},
 138616,
 {'hlk_meta': (138616, 312330, 2458399)})

In [ ]:
train_dset = XCDataset.from_file(**cfg['path']['train'], **cfg['parameters'])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
train_dset.show_data(n=2)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,Microsail,Microsail,"[101, 12702, 15816, 2140, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",[194848],[List_of_sailing_boat_types],[List of sailing boat types],"[[101, 2862, 1997, 8354, 4049, 4127, 102]]","[[0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1]]","[254, 281, 527389, 527392, 883760]","[United_States, France, Gary_Mull, Fractional_rig, Jeanneau]","[United States, France, Gary Mull, Fractional rig, Jeanneau]","[[101, 2142, 2163, 102], [101, 2605, 102], [101, 5639, 14163, 3363, 102], [101, 12884, 2389, 19838, 102], [101, 14537, 4887, 102]]","[[0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1]]","[[43854, 261695, 297082]]","[[National_Collegiate_Athletic_Association, Sacred_Heart_University, Sacred_Heart_Pioneers]]","[[National Collegiate Athletic Association, Sacred Heart University, Sacred Heart Pioneers]]","[[[101, 2120, 9234, 5188, 2523, 102], [101, 6730, 2540, 2118, 102], [101, 6730, 2540, 13200, 102]]]","[[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]]","[[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]]"
1,Captain_Benjamin_Williams_House,Captain Benjamin Williams House,"[101, 2952, 6425, 3766, 2160, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",[235682],"[National_Register_of_Historic_Places_listings_in_Middletown,_Connecticut]","[National Register of Historic Places listings in Middletown, Connecticut]","[[101, 2120, 4236, 1997, 3181, 3182, 26213, 1999, 28747, 1010, 6117, 102]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]","[7140, 38915]","[National_Register_of_Historic_Places, Connecticut_River]","[National Register of Historic Places, Connecticut River]","[[101, 2120, 4236, 1997, 3181, 3182, 102], [101, 6117, 2314, 102]]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]","[[8398, 19705, 26775, 44140, 55989, 64716, 75195, 81592, 129263, 195143, 267626, 281086, 281149, 281203, 281221, 281352, 304511, 316421, 578952, 723622, 742621, 744544, 799315, 844632, 1019297, 1357876, 1357877, 1357878, 1357879, 1357880]]","[[2004_Summer_Olympics, Baseball_at_the_Summer_Olympics, Australians, Run_batted_in, Second_baseman, Maccabi_World_Union, Jewish_Virtual_Library, New_South_Wales_Patriots, Southeastern_Louisiana_University, Sri_Lanka_national_cricket_team, Baseball_at_the_2004_Summer_Olympics, World_Baseball_Classic, Rodney_van_Buizen, Brett_Roneberg, Glenn_Williams, Brendan_Kingman, 2007_Baseball_World_Cup, Greg_Jelks, 2005_Baseball_World_Cup, Judaism_in_Australia, New_Haven_County_Cutters, International_Baseball_League_of_Australia, 2001_Baseball_World_Cup, Northeast_League, Taiwan_Major_League, Masada_College, 2006_Claxton_Shield, 2004_Claxton_Shield, 2007_Claxton_Shield, Catskill_Cougars]]","[[2004 Summer Olympics, Baseball at the Summer Olympics, Australians, Run batted in, Second baseman, Maccabi World Union, Jewish Virtual Library, New South Wales Patriots, Southeastern Louisiana University, Sri Lanka national cricket team, Baseball at the 2004 Summer Olympics, World Baseball Classic, Rodney van Buizen, Brett Roneberg, Glenn Williams, Brendan Kingman, 2007 Baseball World Cup, Greg Jelks, 2005 Baseball World Cup, Judaism in Australia, New Haven County Cutters, International Baseball League of Australia, 2001 Baseball World Cup, Northeast League, Taiwan Major League, Masada College, 2006 Claxton Shield, 2004 Claxton Shield, 2007 Claxton Shield, Catskill Cougars]]","[[[101, 2432, 26

## Collator

In [ ]:
#| export
class XCCollator:

    def __init__(self, tfms):
        self.tfms = tfms

    def __call__(self, x):
        return self.tfms(x)
        

### Example

In [ ]:
tokz = AutoTokenizer.from_pretrained('bert-base-uncased')
tfm_cfg = {
    'sep_tok': tokz.sep_token_id,
    'pad_tok': tokz.pad_token_id,
    'prefix': 'lbl',
    'pad_side': 'right',
    'inp': 'data',
    'targ': 'lbl2data',
    'ptr': 'lbl2data_data2ptr',
    'drop': False,
    'ret_t': True,
    'in_place': False,
    'lev': 0,
    'device': 'cpu',
}

In [ ]:
tfms = TfmPipeline([XCPadFeatTfm(**tfm_cfg), AlignInputIdsTfm(**tfm_cfg)])
collator = XCCollator(tfms)

In [ ]:
b = train_dset.one_batch(10)
o = collator(b)

In [ ]:
o

{'lbl2data_idx': tensor([152270, 152271,  21303,  19438, 259968, 134235, 181495,  22805, 266055,
        138124,  63265, 130051, 195205,  57429,   2567,   2568]), 'lbl2data_identifier': ['Communes_of_the_Alpes-de-Haute-Provence_department', '3=Liste_des_anciennes_communes_des_Alpes-de-Haute-Provence', 'List_of_mountain_peaks_of_the_United_States', 'List_of_mountain_peaks_of_North_America', '2002-03_Heineken_Cup', 'Football_in_Poland', 'PogoÃ\x85Â\x84_Szczecin', 'List_of_national_anthems', 'National_Anthem_of_South_Ossetia', 'VicuÃ\x83Â±a_family', 'Louisiana_African_American_Heritage_Trail', 'Creoles_of_color', 'UK_insolvency_law', 'Communes_of_the_Seine-et-Marne_department', 'List_of_foreign_ministers_in_2017', 'List_of_current_foreign_ministers'], 'lbl2data_input_text': ['Communes of the Alpes-de-Haute-Provence department', '3=Liste des anciennes communes des Alpes-de-Haute-Provence', 'List of mountain peaks of the United States', 'List of mountain peaks of North America', '2002-03 He

In [ ]:
dl = DataLoader(train_dset, batch_size=10, collate_fn=collator)

In [ ]:
b = next(iter(dl))

In [ ]:
b

{'lbl2data_idx': tensor([    0,     2,     3,     9, 26766,    14,    13,    16,    18,    23,
           29,    45,    60,    51,    67,    65,   101,   102]), 'lbl2data_identifier': ['Antinomianism', 'Anarchism', 'Autism', 'Conimbricenses', 'Aristotle', 'List_of_actors_with_Academy_Award_nominations', 'List_of_Academy_Award_records', 'Clock_synchronization', 'Precision_Time_Protocol', 'Egotism', 'Inclusive_fitness', 'Canadian_pioneers_in_early_Hollywood', 'Qualitative_research', 'Ethnobiology', 'History_of_agricultural_science', 'American_Society_of_Agronomy', '12_basic_principles_of_animation', 'Anime'], 'lbl2data_input_text': ['Antinomianism', 'Anarchism', 'Autism', 'Conimbricenses', 'Aristotle', 'List of actors with Academy Award nominations', 'List of Academy Award records', 'Clock synchronization', 'Precision Time Protocol', 'Egotism', 'Inclusive fitness', 'Canadian pioneers in early Hollywood', 'Qualitative research', 'Ethnobiology', 'History of agricultural science', 'American

In [ ]:
b.keys()

dict_keys(['lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask', 'hlk2lbl2data_idx', 'hlk2lbl2data_identifier', 'hlk2lbl2data_input_text', 'hlk2lbl2data_input_ids', 'hlk2lbl2data_token_type_ids', 'hlk2lbl2data_attention_mask', 'hlk2lbl2data_data2ptr', 'hlk2lbl2data_lbl2ptr', 'hlk2data_idx', 'hlk2data_identifier', 'hlk2data_input_text', 'hlk2data_input_ids', 'hlk2data_token_type_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr'])

## Configuration

In [ ]:
import json

In [ ]:
cfg = {
    'path':{
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_trn_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt',
            },
        },
        'test': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/tst_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/test.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_test.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_tst_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt',
            },
        },
    },
    'parameters': {
        'cols': ['identifier', 'input_text'],
        'use_tokz': True,
        'tokz': 'bert-base-uncased',
        'fld': 'input_text',
        'max_len': 32,
        'pad_side': 'right',
        'inp': 'data',
        'targ': 'lbl2data',
        'ptr': 'lbl2data_data-ptr',
        'drop': False,
        'ret_t': True,
        'in_place': False,
        'lev': 0,
        'device': 'cpu',
    },
}

In [ ]:
cfg_dir = '/home/scai/phd/aiz218323/scratch/Projects/xcai/cfg/'

In [ ]:
with open(f'{cfg_dir}/data.json', 'w') as f:
    json.dump(cfg, f, indent=2)

## Data block

### BaseXCDataBlock

In [ ]:
#| export
class BaseXCDataBlock:

    @delegates(DataLoader.__init__)
    def __init__(self, 
                 dset:XCDataset, 
                 collate_fn:Callable=None,
                 **kwargs):
        self.dset, self.dl_kwargs, self.collate_fn = dset, self._get_dl_kwargs(**kwargs), collate_fn
        self.dl = DataLoader(dset, collate_fn=collate_fn, **self.dl_kwargs) if collate_fn is not None else None

    @classmethod
    @delegates(XCDataset.from_file)
    def from_file(cls, collate_fn:Callable=None, **kwargs):
        return BaseXCDataBlock(XCDataset.from_file(**kwargs), collate_fn, **kwargs)

    def __len__(self):
        return len(self.dset)

    def _get_dl_kwargs(self, **kwargs):
        dl_params = inspect.signature(DataLoader.__init__).parameters
        return {k:v for k,v in kwargs.items() if k in dl_params}

    
    def _getitems(self, idxs:List):
        return BaseXCDataBlock(self.dset._getitems(idxs), collate_fn=self.collate_fn, **self.dl_kwargs)

    @property
    def bsz(self): return self.dl.batch_size

    @bsz.setter
    def bsz(self, v):
        self.dl_kwargs['batch_size'] = v
        self.dl = DataLoader(self.dset, collate_fn=self.collate_fn, **self.dl_kwargs) if self.collate_fn is not None else None

    @property
    def data_lbl_filterer(self): return self.dset.data.data_lbl_filterer

    @data_lbl_filterer.setter
    def data_lbl_filterer(self, val): self.dset.data.data_lbl_filterer = val

    @typedispatch
    def one_batch(self):
        return next(iter(self.dl))

    @typedispatch
    def one_batch(self, bsz:int):
        self.dl_kwargs['batch_size'] = bsz
        self.dl = DataLoader(self.dset, collate_fn=self.collate_fn, **self.dl_kwargs) if self.collate_fn is not None else None
        return next(iter(self.dl))
        
        

In [ ]:
#| export
@patch
def filterer(cls:BaseXCDataBlock, train:'BaseXCDataBlock', valid:'BaseXCDataBlock', fld:Optional[str]='identifier'):
    train_info, valid_info, lbl_info = train.dset.data.data_info, valid.dset.data.data_info, train.dset.data.lbl_info
    if fld not in train_info: raise ValueError(f'`{fld}` not in `data_info`')
        
    train.data_lbl_filterer, valid_filterer = Filterer.generate(train_info[fld], valid_info[fld], lbl_info[fld], 
                                                                train.dset.data.data_lbl, valid.dset.data.data_lbl)
    _, valid_filterer, idx = Filterer.prune(valid.dset.data.data_lbl, valid_filterer)
    
    valid = valid._getitems(idx)
    valid.data_lbl_filterer = valid_filterer
    
    return train, valid

@patch
def splitter(cls:BaseXCDataBlock, valid_pct:Optional[float]=0.2, seed=None):
    if seed is not None: torch.manual_seed(seed)
    rnd_idx = list(torch.randperm(len(cls)).numpy())
    cut = int(valid_pct * len(cls))
    train, valid = cls._getitems(rnd_idx[cut:]), cls._getitems(rnd_idx[:cut])
    return cls.filterer(train, valid)
        

#### Example

In [ ]:
train_block = BaseXCDataBlock(train_dset, batch_size=2, collate_fn=collator)

In [ ]:
split1, split2 = train_block.splitter()
print('split 1:',split1.dset.n_data,split1.dset.n_lbl,split1.dset.n_meta)
print('split 2:',split2.dset.n_data,split2.dset.n_lbl,split2.dset.n_meta) 

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


split 1: 554466 312330 2458399
split 2: 137588 312330 2458399


In [ ]:
b = train_block.one_batch()

In [ ]:
b = train_block.one_batch(20)

In [ ]:
train_block = BaseXCDataBlock.from_file(**cfg['path']['train'], **cfg['parameters'], collate_fn=collator)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
next(iter(train_block.dl))

{'lbl2data_idx': tensor([0, 1, 2]), 'lbl2data_identifier': ['Antinomianism', 'Libertarian_socialism', 'Anarchism'], 'lbl2data_input_text': ['Antinomianism', 'Libertarian socialism', 'Anarchism'], 'lbl2data_input_ids': tensor([[  101,  3424,  3630, 20924,   102,     0],
        [  101, 19297, 14649,   102,     0,     0],
        [  101,  9617, 11140,  2964,   102,     0]]), 'lbl2data_token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'lbl2data_attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0]]), 'lbl2data_data2ptr': tensor([3]), 'data_identifier': ['Anarchism'], 'data_input_text': ['Anarchism'], 'data_input_ids': tensor([[  101,  9617, 11140,  2964,   102]]), 'data_token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'data_attention_mask': tensor([[1, 1, 1, 1, 1]]), 'hlk2lbl2data_idx': tensor([   1992,    4612,    4697,    4706,    4709,    4892,    5256,    7587,
           9813,   10948,   110

### XCDataBlock

In [ ]:
#| export
class XCDataBlock:

    def __init__(self, train:BaseXCDataBlock=None, valid:BaseXCDataBlock=None, test:BaseXCDataBlock=None):
        self.train, self.valid, self.test = train, valid, test

    @staticmethod
    def load_cfg(fname):
        with open(fname, 'r') as f: return json.load(f)

    @property
    def lbl_info(self): return self.train.dset.data.lbl_info

    @property
    def n_lbl(self): return self.train.dset.n_lbl

    @property
    def collator(self): return self.train.collate_fn
        
    @classmethod
    def from_cfg(cls, 
                 cfg:Union[str,Dict],
                 collate_fn:Optional[Callable]=None,
                 valid_pct:Optional[float]=0.2,
                 seed=None):
        if isinstance(cfg, str): cfg = cls.load_cfg(cfg)
        blks = {o:BaseXCDataBlock.from_file(**cfg['path'][o], **cfg['parameters'], collate_fn=collate_fn) for o in ['train', 'valid', 'test'] if o in cfg['path']}
        if 'valid' not in blks: blks['train'], blks['valid'] = blks['train'].splitter(valid_pct, seed=seed)
        return cls(**blks)
        

#### Example

In [ ]:
block = XCDataBlock(train=train_block)

In [ ]:
lbl_info = block.lbl_info

In [ ]:
lbl_info['input_ids'][:10]

[[101, 3424, 3630, 20924, 2964, 102],
 [101, 19297, 14649, 102],
 [101, 9617, 11140, 2964, 102],
 [101, 19465, 102],
 [101, 13798, 1997, 8181, 5367, 102],
 [101, 7734, 2162, 1997, 6889, 102],
 [101, 4519, 2793, 5349, 102],
 [101, 2862, 1997, 7008, 1997, 8181, 5367, 102],
 [101, 2862, 1997, 2942, 2916, 4177, 102],
 [101, 9530, 5714, 23736, 19023, 2229, 102]]

In [ ]:
%time block = XCDataBlock.from_cfg(cfg, collate_fn=collator)

CPU times: user 1min 18s, sys: 930 ms, total: 1min 19s
Wall time: 44.2 s


In [ ]:
print('n_data :', f'{block.train.dset.n_data},{block.valid.dset.n_data},{block.test.dset.n_data}')
print('n_lbl  :', f'{block.train.dset.n_lbl},{block.valid.dset.n_lbl},{block.test.dset.n_lbl}')
print('n_meta :', f'{block.train.dset.n_meta},{block.valid.dset.n_meta},{block.test.dset.n_meta}')

n_data : 554466,138616,177515
n_lbl  : 312330,312330,312330
n_meta : None,None,None


In [ ]:
block.valid.dset.show_data(n=2)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,2019_Caribbean_Club_Championship,2019 Caribbean Club Championship,"[101, 10476, 7139, 2252, 2528, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]","[310617, 310618]","[2019_CONCACAF_League, 2020_CONCACAF_Champions_League]","[2019 CONCACAF League, 2020 CONCACAF Champions League]","[[101, 10476, 22169, 2223, 102], [101, 12609, 22169, 3966, 2223, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]","[2932, 4843, 8132, 115570, 122131, 147896, 158253, 158260, 246100, 286923, 289314, 304800, 350441, 403932, 416728, 416741, 416746, 416750, 416921, 416926, 416929, 416930, 416945, 416949, 416954, 416961, 427446, 427447, 427448, 427449, 427450, 427451, 427452, 427453, 427454, 427455, 427456, 427457, 427458, 427459, 427460, 427461]","[California, San_Francisco, Eastern_Time_Zone, Willemstad, CONCACAF, Jamaica_Football_Federation, Football_Association_of_Cuba, Bermuda_Football_Association, Caribbean_Football_Union, Surinamese_Football_Association, Guyana_Football_Federation, Barbados_Football_Association, Caribbean_Club_Championship, Bahamas_Football_Association, Cibao_FC, Grenada_Football_Association, Club_Franciscain, Ligue_de_football_de_la_Martinique, Weymouth_Wales_FC, Anguilla_Football_Association, Aruba_Football_Federation, SV_Real_Rincon, Bonaire_Football_Federation, Dominica_Football_Association, Guadeloupean_League_of_Football, Montserrat_Football_Association, 2017_TT_Pro_League, AS_Capoise, 2018_Caribbean_Club_Championship, SV_Dakota, Fruta_Conquerors_FC, FC_Santiago_de_Cuba, Platinum_FC, 2018_SLFA_First_Division, Village_Superstars_FC, 2020_Caribbean_Club_Championship, Ergilio_Hato_Stadium, 2018_Campeonato_Nacional_de_FÃºtbol_de_Cuba, 2018_Barbados_Premier_League, Scholars_International_SC, CRKSV_Jong_Holland, CS_Moulien]","[California, San Francisco, Eastern Time Zone, Willemstad, CONCACAF, Jamaica Football Federation, Football Association of Cuba, Bermuda Football Association, Caribbean Football Union, Surinamese Football Association, Guyana Football Federation, Barbados Football Association, Caribbean Club Championship, Bahamas Football Association, Cibao FC, Grenada Football Association, Club Franciscain, Ligue de football de la Martinique, Weymouth Wales FC, Anguilla Football Association, Aruba Football Federation, SV Real Rincon, Bonaire Football Federation, Dominica Football Association, Guadeloupean League of Football, Montserrat Football Association, 2017 TT Pro League, AS Capoise, 2018 Caribbean Club Championship, SV Dakota, Fruta Conquerors FC, FC Santiago de Cuba, Platinum FC, 2018 SLFA First Division, Village Superstars FC, 2020 Caribbean Club Championship, Ergilio Hato Stadium, 2018 Campeonato Nacional de FÃºtbol de Cuba, 2018 Barbados Premier League, Scholars International SC, CRKSV Jong Holland, CS Moulien]","[[101, 2662, 102], [101, 2624, 3799, 102], [101, 2789, 2051, 4224, 102], [101, 18811, 16917, 102], [101, 22169, 102], [101, 9156, 2374, 4657, 102], [101, 2374, 2523, 1997, 7394, 102], [101, 13525, 2374, 2523, 102], [101, 7139, 2374, 2586, 102], [101, 25050, 3366, 2374, 2523, 102], [101, 18786, 2374, 4657, 102], [101, 16893, 2374, 2523, 102], [101, 7139, 2252, 2528, 102], [101, 17094, 2374, 2523, 102], [101, 25022, 3676, 2080, 4429, 102], [101, 29153, 2374, 2523, 102], [101, 2252, 4557, 3540, 2378, 102], [101, 18374, 2139, 2374, 2139, 2474, 29365, 102], [101, 2057, 24335, 17167, 3575, 4429, 102], [101, 17076, 19231, 2721, 2374, 2523, 102], [101, 12098, 19761, 2374, 4657, 102], [101, 17917, 2613, 15544, 15305, 

In [ ]:
train_lbl = block.train.dset.data.data_lbl
test_lbl = block.valid.dset.data.data_lbl

In [ ]:
train_identifier = block.train.dset.data.data_info['identifier']
test_identifier = block.valid.dset.data.data_info['identifier']
lbl_identifier = block.train.dset.data.lbl_info['identifier']

In [ ]:
x, y = gen_filterer(train_identifier, test_identifier, lbl_identifier, train_lbl, test_lbl)